# Add Info to JSON Files
## Authors: Ganesh Arvapalli and Patrick Myers

This file loads in a JSON file and the corresponding nifti image. Important information about the image is added to the JSON file

In [2]:
from os import listdir
from os.path import isfile, join
import numpy as np
from nibabel.testing import data_path
import nibabel as nib
import json
import pdb

## Adds the size of each parcel in an atlas to that JSON file

Flattens the nifti image into a 1D array. Then puts the data into a 2D list where each row, i, contains the locations of the voxels in the ith parcel. The size of each of these lists is calculated. This information is added to the corresponding key in the JSON file

In [16]:
def Size_JSON(json_file, nifti_file, outname):    
    img = nib.load(nifti_file)
    img_data = img.get_fdata()
    img_flat = img_data.flatten()
    img_flat.tolist()
    img_flat = [int(i) for i in img_flat]
    values = np.unique(img_flat)
    counts = np.array(range(1, len(img_flat)+1))
    counts = [int(i) for i in counts]
    img_flatten = list(zip(img_flat, counts))
    img_flatten = sorted(img_flatten, key = lambda l:l[0])

    img_cell = [[] for i in range(len(values))]
    start = 0
    for i in range(0,len(values)):
        img_list = [x[0] for x in img_flatten]
        end = max(loc for loc, val in enumerate(img_list) if val == values[i])
        for j in range(start,end):
            img_cell[i].append(img_flatten[j][1])  
        start = end+1
    
    with open(json_file) as f:
        data = json.load(f)
    
    for i in range(0,len(values)):
        data[str(i)]["size"] = str(len(img_cell[i]))+' voxels'
    
    with open(outname, 'w') as f:
        json.dump(data, f, indent=4, sort_keys=True)


### Run for every JSON/Nifti pair

Not every Nifti file has a corresponding JSON pair, so the JSON files were used as the key in the search for the corresponding Nifti file. Specifies the output JSON to a new folder to not get confused.

In [21]:
JSON_files = listdir('../JSON_files')
files = [f.split('.')[0] for f in JSON_files]
files = [i for i in files if not i.startswith("DS")]
JSON_path = '../JSON_files/'
JSON_extension = '.json'
Nifti_path = '../Nifti_images/'
Nifti_extension = '.nii.gz'
outfile_path = '../Updated_JSON_files/'

for file in files:
    json_file = JSON_path+file+JSON_extension
    print(json_file)
    nifti_file = Nifti_path+file+Nifti_extension
    outfile = outfile_path+file+'_updated'+JSON_extension
    Size_JSON(json_file,nifti_file,outfile)




../JSON_files/DKT_space-MNI152NLin6_res-1x1x1.json
../JSON_files/CPAC200_space-MNI152NLin6_res-1x1x1.json
../JSON_files/JHU_space-MNI152NLin6_res-1x1x1.json
